In [5]:
#install dependency
!pip install protein-bert
!pip install gdown
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 20.5 MB/s 


In [3]:
!gdown --folder '1MMvRjjeR1Hz6hw08WZhZwLI-89pUpYtc'

Retrieving folder list
Processing file 1OEdb0koLSS4PMoswa8ie7zuQzA3qXflC fine_tuned_model_other_fold_0.h5
Processing file 17Qt7inYBWXqElmWUKSCXduWWawjaSAJN fine_tuned_model_other_fold_1.h5
Processing file 1NzJ5IEtAhSffSSVDMcFmA8xQUVkphKda fine_tuned_model_other_fold_2.h5
Processing file 1_-KYtcWgN8H_SLvOp_scFn28Yf1U1yEp fine_tuned_model_other_fold_3.h5
Processing file 1Xr7wbPTeB13gZnszj0whbo8rVxApPZBN fine_tuned_model_other_fold_4.h5
Processing file 1T3Go16f8SOwzSurB47qdPIuAOrmQCMSU label_to_family.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1OEdb0koLSS4PMoswa8ie7zuQzA3qXflC
To: /content/GT_MODELS/fine_tuned_model_other_fold_0.h5
100% 71.1M/71.1M [00:00<00:00, 77.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=17Qt7inYBWXqElmWUKSCXduWWawjaSAJN
To: /content/GT_MODELS/fine_tuned_model_other_fold_1.h5
100% 71.1M/71.1M [00:00<00:00, 104MB/s]
Downloading...
From: https

In [6]:
import os
import tensorflow as tf
from Bio import SeqIO
import os
import pandas as pd
import os
import pandas as pd
from IPython.display import display
from tensorflow import keras
from sklearn.model_selection import train_test_split
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
import numpy as np
from proteinbert.finetuning import encode_categorical_Y
from proteinbert.finetuning import encode_Y
from proteinbert.finetuning import encode_dataset

In [7]:
#load the predictors
def get_models(size = 1024):
    df = pd.read_csv('GT_MODELS/label_to_family.csv',index_col=[0])
    unique_labels = sorted(set(df['label'].unique()))
    label_dict = dict(zip(df['label'],df['FamilyId']))
    OUTPUT_TYPE = OutputType(False, 'categorical')
    OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, unique_labels)

    pretrained_model_generator, input_encoder = load_pretrained_model(
    local_model_dump_dir='',
    validate_downloading=False)

    # get_model_with_hidden_layers_as_outputs gives the model output access to the hidden layers (on top of the output)
    model_generator = FinetuningModelGenerator(
        pretrained_model_generator, 
        OUTPUT_SPEC, 
        pretraining_model_manipulation_function = get_model_with_hidden_layers_as_outputs,
        dropout_rate = 0.5)


    fold=0
    fine_tuned_model_0 = model_generator.create_model(size)
    fine_tuned_model_0.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=1
    fine_tuned_model_1 = model_generator.create_model(size)
    fine_tuned_model_1.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=2
    fine_tuned_model_2 = model_generator.create_model(size)
    fine_tuned_model_2.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=3
    fine_tuned_model_3 = model_generator.create_model(size)
    fine_tuned_model_3.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=4
    fine_tuned_model_4 = model_generator.create_model(size)
    fine_tuned_model_4.load_weights('GT_MODELS/fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))    

    models = [
        fine_tuned_model_0,
        fine_tuned_model_1,
        fine_tuned_model_2,
        fine_tuned_model_3,
        fine_tuned_model_4,
    ]
    return models, input_encoder, OUTPUT_SPEC, label_dict

models, input_encoder, OUTPUT_SPEC, label_dict = get_models()

Downloaded file: epoch_92400_sample_23500000.pkl
Created: default.pkl


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
seq = [
       'A'*300
]

target = [0]

In [10]:
#temp=temp.reset_index(drop=True)
X, y_true, sample_weights = encode_dataset(seq, target, input_encoder, OUTPUT_SPEC, \
                seq_len = 512*2, needs_filtering = False)

In [13]:
in_df = pd.DataFrame()
in_df['seq']=seq

In [14]:
from tqdm.auto import tqdm
for index, model in tqdm(enumerate(models)):
    preds = model.predict(X, batch_size = 32)
    index_sorted = np.argsort(preds, axis=1)
    
    labels_1 = pd.Series(index_sorted[:,-1])
    labels_2 = pd.Series(index_sorted[:,-2])
    
    family_1 = labels_1.map(label_dict)
    family_2 = labels_2.map(label_dict)
    
    prob_1 = preds[np.arange(len(X[0])), labels_1]
    prob_2 = preds[np.arange(len(X[0])), labels_2]
    
    in_df['model_'+str(index)+'_labels_1']=labels_1.values
    in_df['model_'+str(index)+'_family_1']=family_1.values
    in_df['model_'+str(index)+'_prob_1']=prob_1
    
    #list_df_500['model_'+str(index)+'_labels_2']=labels_2
    #list_df_500['model_'+str(index)+'_family_2']=family_2
    #list_df_500['model_'+str(index)+'_prob_2']=prob_2
    
    #temp['preds_model_'+str(index)]

0it [00:00, ?it/s]

In [15]:
in_df.head()

,seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,model_2_family_1,model_2_prob_1,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,110,Other,1.0,110,Other,1.0,110,Other,1.0,110,Other,1.0,110,Other,1.0


In [16]:
def select(X):
    temp = []
    items = list(set(X))
    to_count = list(X)
    #print(temp,items,to_count)
    for n in items:
        temp.append((to_count.count(n),n))
    temp.sort()
    #print(temp)
    return temp[-1][-1]


def vote_mean(X):
    #print(X)
    majority = X['vote']
    score = []
    for a,b in zip(X[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']],
                   X[['model_0_prob_1','model_1_prob_1','model_2_prob_1','model_3_prob_1','model_4_prob_1']]):
        if a==majority:
            score.append(b)
    return np.mean(score)
                   


def get_dict_from_desc(X):
    temp_dict = {}
    item_list= X.split('|')
    #print(item_list)
    for item in item_list:
        if '=' in item:
            key = item.split('=')[0].strip()
            value = item.split('=')[1].strip()
            temp_dict[key]=value
            #print(key,value)
    return temp_dict

def get_desc(X, target):
    temp_dict = get_dict_from_desc(X)
    #print(temp_dict)
    return temp_dict[target]


In [17]:
in_df['vote']=in_df[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']].apply(select,axis=1)
in_df['vote_mean']=in_df.apply(vote_mean,axis=1)

,seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,model_2_family_1,model_2_prob_1,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,vote_mean
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,110,Other,1.0,110,Other,1.0,110,Other,1.0,110,Other,1.0,110,Other,1.0,Other,1.0


In [18]:
in_df[['vote','vote_mean']]

,vote,vote_mean
0,Other,1.0
